In [1]:
import pandas as pd
import wandb
import keras
from wandb.keras import WandbCallback
import shutil
import tensorflow as tf
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

Using TensorFlow backend.


#### Setup network architecture

In [2]:
# lr: float >= 0. Learning rate.
# beta_1: float, 0 < beta < 1. Generally close to 1.
# beta_2: float, 0 < beta < 1. Generally close to 1.
# epsilon: float >= 0. Fuzz factor.
# decay: float >= 0. Learning rate decay over each update.
##Default params
# lr=0.001,
# beta_1=0.9,
# beta_2=0.999,
# epsilon=1e-08,
# decay=0.0,

hyper_params = dict(
    image_size = 224,
    num_layers_frozen = 4 + 5*6,
    num_layers_pop = 6*6,
    dense_1_size = 1024,
    dense_2_size = 512,
    dense_3_size = 128,
    batch_size = 32,
#     steps_per_epoch = 200,
    #Adam Optimizer
    lr=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-08,
    decay=0.000,
    epochs = 5
)

wandb.init(project="car_nanodegree_capstone", config=hyper_params)

config = wandb.config

dataBaseFolder = '../../../../data/'
# datasdcnd = dataBaseFolder + 'dataset-sdcnd-capstone/data/real_training_data/'
dataTL = dataBaseFolder + 'tl_engineer5/'

base_model = keras.applications.mobilenet.MobileNet(input_shape=(config.image_size,config.image_size,3), 
                                                    include_top=False, weights='imagenet')

for layer in base_model.layers[:config.num_layers_frozen]:
    layer.trainable=False

for layer in base_model.layers[config.num_layers_frozen:]:
    layer.trainable=True

x = base_model.layers[-config.num_layers_pop-1].output
x = GlobalAveragePooling2D()(x)
# we add dense layers so that the model can learn more complex functions and classify for better results.
x = Dense(config.dense_1_size,activation='relu')(x) 
x = Dense(config.dense_2_size,activation='relu')(x) #dense layer 2
x = Dense(config.dense_3_size,activation='relu')(x) #dense layer 3
preds = Dense(4,activation='softmax')(x) #final layer with softmax activation

model = Model(inputs=base_model.input, outputs=preds)

adam_opt = Adam(lr=config.lr, beta_1=config.beta_1, beta_2=config.beta_2, epsilon=config.epsilon, decay=config.decay)

model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ········


In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 32)      0         
__________

In [4]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    dataTL+'test',
    target_size=(config.image_size, config.image_size),
    batch_size=1,
)

train_generator = train_datagen.flow_from_directory(
        dataTL+'train',
        target_size=(config.image_size, config.image_size),
        batch_size=config.batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        dataTL+'val',
        target_size=(config.image_size, config.image_size),
        batch_size=config.batch_size,
        class_mode='categorical')

model.fit_generator(
        train_generator,
        epochs=config.epochs,
        steps_per_epoch=3521 // config.batch_size,
        validation_data=validation_generator,
        validation_steps = 755//config.batch_size,
        callbacks=[WandbCallback()])


model.save(os.path.join(wandb.run.dir, "model_34_frozen_36_pop.h5"))

saver = tf.train.Saver()
saver.save(K.get_session(), '/tmp/keras_model.ckpt')

Found 627 images belonging to 4 classes.
Found 2917 images belonging to 4 classes.
Found 625 images belonging to 4 classes.
Epoch 1/5
110/110 [==============================] - 175s - loss: 0.6169 - acc: 0.7733 - val_loss: 0.5987 - val_acc: 0.7878
Epoch 2/5
110/110 [==============================] - 190s - loss: 0.3806 - acc: 0.8704 - val_loss: 0.4046 - val_acc: 0.8724
Epoch 3/5
110/110 [==============================] - 193s - loss: 0.3020 - acc: 0.8946 - val_loss: 0.2041 - val_acc: 0.9376
Epoch 4/5
110/110 [==============================] - 183s - loss: 0.1675 - acc: 0.9443 - val_loss: 0.1190 - val_acc: 0.9518
Epoch 5/5
110/110 [==============================] - 167s - loss: 0.1851 - acc: 0.9398 - val_loss: 0.1398 - val_acc: 0.9556


'/tmp/keras_model.ckpt'

In [5]:
scores = model.evaluate_generator(generator=test_generator, steps=755//config.batch_size, workers=1)
print('Accuracy: ', scores[1])

filenames = test_generator.filenames
nb_samples = len(filenames)
pred = model.predict_generator(test_generator, steps=nb_samples, verbose=1, workers=1)
predicted_class_indices = np.argmax(pred, axis=1)

#Confution Matrix and Classification Report
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, predicted_class_indices))

# labels = (train_generator.class_indices)
# labels = dict((v,k) for k,v in labels.items())
# predictions = [labels[k] for k in predicted_class_indices]

print('Classification Report')
print(classification_report(test_generator.classes, predicted_class_indices))

# filenames=test_generator.filenames
# results=pd.DataFrame({"Filename":filenames,
#                       "Predictions":predictions})

# results.to_csv(dataBaseFolder + 'results_baseline.csv',index=False)

print(train_generator.class_indices)
print(validation_generator.class_indices)

('Accuracy: ', 1.0)
627/627 [==============================] - 26s    
Confusion Matrix
[[139 126  26]
 [142 119  20]
 [ 28  19   8]]
Classification Report
              precision    recall  f1-score   support

           1       0.45      0.48      0.46       291
           2       0.45      0.42      0.44       281
           3       0.15      0.15      0.15        55

   micro avg       0.42      0.42      0.42       627
   macro avg       0.35      0.35      0.35       627
weighted avg       0.42      0.42      0.42       627

{'unknown': 2, 'green': 0, 'yellow': 3, 'red': 1}
{'unknown': 2, 'green': 0, 'yellow': 3, 'red': 1}


In [6]:
import cv2

In [7]:
def predict_img(model, fname):
    cv_image = cv2.imread(fname)
    cv_image = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB)
    cv_image = cv2.resize(cv_image, (224, 224))
    cv_image = cv_image/255.
    image_expanded = np.expand_dims(cv_image, axis=0)
    pred = model.predict(image_expanded)
    print(pred)
    return np.argmax(pred, axis=1)

In [8]:
pred_class_arr = []
for img_name in test_generator.filenames:
    pred_class = predict_img(model, os.path.join(dataTL+'test', img_name))
    print(img_name, pred_class)
    pred_class_arr.append(pred_class[0])

[[  4.50046234e-16   9.99999762e-01   5.91923266e-09   1.88745631e-07]]
('red/out00206.png', array([1]))
[[  3.96302809e-13   9.99995112e-01   2.01993643e-07   4.65649509e-06]]
('red/out05356.png', array([1]))
[[  6.06948825e-10   9.99858379e-01   1.10966703e-05   1.30560773e-04]]
('red/out08114.png', array([1]))
[[  9.31334938e-08   9.98669505e-01   1.94970984e-04   1.13551959e-03]]
('red/out08100.png', array([1]))
[[  2.04648473e-12   9.99988914e-01   4.89834918e-07   1.05897880e-05]]
('red/out07582.png', array([1]))
[[  1.26728961e-10   9.99939442e-01   5.11933467e-06   5.54205290e-05]]
('red/out01440.png', array([1]))
[[  2.92476687e-09   9.99721587e-01   2.65932558e-05   2.51777121e-04]]
('red/out07964.png', array([1]))
[[  2.31835124e-12   9.99988675e-01   5.22931373e-07   1.08456306e-05]]
('red/out04672.png', array([1]))
[[  1.36429048e-06   9.95251536e-01   8.62032466e-04   3.88506614e-03]]
('red/out05236.png', array([1]))
[[  7.74510144e-12   9.99981880e-01   1.03018579e-06   

[[  7.30574320e-14   9.99997616e-01   8.17370918e-08   2.22359245e-06]]
('red/out00066.png', array([1]))
[[  8.73361206e-09   9.99462903e-01   4.57841452e-05   4.91336279e-04]]
('red/out00714.png', array([1]))
[[  2.27058362e-14   9.99998569e-01   4.24867785e-08   1.43461352e-06]]
('red/out06826.png', array([1]))
[[  1.96591254e-09   9.99737322e-01   2.14051652e-05   2.41250629e-04]]
('red/out01622.png', array([1]))
[[  7.45504783e-15   9.99999285e-01   2.55655976e-08   7.28068699e-07]]
('red/out03784.png', array([1]))
[[  6.63245237e-09   9.99645352e-01   5.08786397e-05   3.03769251e-04]]
('red/out05120.png', array([1]))
[[  1.12120624e-09   9.99858141e-01   1.89721868e-05   1.22850994e-04]]
('red/out03586.png', array([1]))
[[  3.05444246e-06   9.92321193e-01   1.20419241e-03   6.47162274e-03]]
('red/out05732.png', array([1]))
[[  3.31089378e-10   9.99904275e-01   8.39043878e-06   8.73496174e-05]]
('red/out09108.png', array([1]))
[[  1.59757761e-07   9.98068273e-01   2.40531095e-04   

[[  9.76646675e-07   9.95736599e-01   7.11735804e-04   3.55074648e-03]]
('red/out04828.png', array([1]))
[[  7.85678447e-17   9.99999881e-01   2.35674125e-09   8.33880378e-08]]
('red/out00482.png', array([1]))
[[  1.37960017e-08   9.99505639e-01   7.43444107e-05   4.20042139e-04]]
('red/out06780.png', array([1]))
[[  1.41122204e-04   9.29099679e-01   4.35844138e-02   2.71747876e-02]]
('red/out01776.png', array([1]))
[[  5.05095748e-13   9.99995589e-01   2.77024782e-07   4.12171175e-06]]
('red/out08752.png', array([1]))
[[  2.01439059e-16   9.99999881e-01   3.82202092e-09   1.41036210e-07]]
('red/out04626.png', array([1]))
[[  1.37880707e-10   9.99927163e-01   4.99429279e-06   6.78587094e-05]]
('red/out02096.png', array([1]))
[[  9.11048648e-15   9.99999285e-01   3.00228535e-08   6.97074711e-07]]
('red/out06596.png', array([1]))
[[  9.09281002e-17   9.99999881e-01   2.53003041e-09   9.01031072e-08]]
('red/out00642.png', array([1]))
[[  3.55253934e-12   9.99987602e-01   6.97517407e-07   

[[  1.58153005e-15   9.99999642e-01   1.10857812e-08   3.54799994e-07]]
('red/out00224.png', array([1]))
[[  2.44989641e-12   9.99988914e-01   5.38851339e-07   1.05359049e-05]]
('red/out03088.png', array([1]))
[[  7.39315347e-13   9.99992609e-01   2.68911094e-07   7.18254660e-06]]
('red/out06872.png', array([1]))
[[  6.27466083e-14   9.99997735e-01   7.54040883e-08   2.11991005e-06]]
('red/out04254.png', array([1]))
[[  2.37113020e-12   9.99988556e-01   5.30028728e-07   1.09242528e-05]]
('red/out09016.png', array([1]))
[[  2.88479324e-10   9.99899507e-01   7.54278290e-06   9.30183960e-05]]
('red/out01258.png', array([1]))
[[  1.16054339e-08   9.99371707e-01   5.16645341e-05   5.76601131e-04]]
('red/out08254.png', array([1]))
[[  9.19917896e-13   9.99993205e-01   3.38177955e-07   6.48789137e-06]]
('red/out06286.png', array([1]))
[[  9.28130836e-13   9.99993205e-01   3.40102360e-07   6.49582216e-06]]
('red/out06292.png', array([1]))
[[  5.87107148e-04   8.75149786e-01   7.09343031e-02   

[[  1.46947139e-08   3.51793761e-03   9.96308506e-01   1.73576904e-04]]
('unknown/out03490.png', array([2]))
[[  2.48414017e-06   2.44872905e-02   9.73093867e-01   2.41642399e-03]]
('unknown/out07184.png', array([2]))
[[  2.54399842e-04   1.16087765e-01   8.57451200e-01   2.62066498e-02]]
('unknown/out07806.png', array([2]))
[[  2.41253165e-05   3.80624793e-02   9.54746485e-01   7.16694165e-03]]
('unknown/out01136.png', array([2]))
[[  7.54728902e-08   4.51387372e-03   9.95125234e-01   3.60826409e-04]]
('unknown/out00758.png', array([2]))
[[  2.67557862e-05   5.88737689e-02   9.32886958e-01   8.21252167e-03]]
('unknown/out03292.png', array([2]))
[[  1.74237157e-05   4.99234311e-02   9.43350255e-01   6.70893304e-03]]
('unknown/out07180.png', array([2]))
[[  5.23097850e-11   4.47889848e-04   9.99542356e-01   9.77651689e-06]]
('unknown/out00360.png', array([2]))
[[  1.71669206e-04   9.50729311e-01   1.60948262e-02   3.30041870e-02]]
('unknown/out05218.png', array([1]))
[[  4.18117807e-08 

[[  1.26173745e-05   4.07183468e-02   9.53805208e-01   5.46382368e-03]]
('unknown/out06362.png', array([2]))
[[  4.55624977e-04   1.61980927e-01   8.01352680e-01   3.62107679e-02]]
('unknown/out05680.png', array([2]))
[[  1.18304051e-04   9.52367485e-01   2.24613771e-02   2.50528101e-02]]
('unknown/out02716.png', array([1]))
[[  1.86640988e-04   1.38729975e-01   8.36538315e-01   2.45450437e-02]]
('unknown/out05904.png', array([2]))
[[  6.20278107e-10   5.73218684e-04   9.99397635e-01   2.91247597e-05]]
('unknown/out01948.png', array([2]))
[[  2.31849888e-04   1.36696950e-01   8.37099016e-01   2.59721614e-02]]
('unknown/out07486.png', array([2]))
[[  2.91555625e-06   2.11146250e-02   9.76418793e-01   2.46366928e-03]]
('unknown/out03186.png', array([2]))
[[  2.09341364e-10   4.55297733e-04   9.99528170e-01   1.65077672e-05]]
('unknown/out08038.png', array([2]))
[[  3.27421779e-09   1.33033853e-03   9.98599827e-01   6.98484582e-05]]
('unknown/out00100.png', array([2]))
[[ 0.00074912  0.24

[[  1.92488088e-08   2.86758435e-03   9.96951938e-01   1.80422416e-04]]
('unknown/out00096.png', array([2]))
[[  3.29643761e-08   3.51471873e-03   9.96251643e-01   2.33627215e-04]]
('unknown/out02656.png', array([2]))
[[  1.83879172e-06   2.02190075e-02   9.77763295e-01   2.01578578e-03]]
('unknown/out03206.png', array([2]))
[[  4.04648870e-10   4.96538822e-04   9.99480188e-01   2.32798993e-05]]
('unknown/out00280.png', array([2]))
[[  2.65146412e-11   2.06948826e-04   9.99787271e-01   5.80081905e-06]]
('unknown/out04958.png', array([2]))
[[  5.23446850e-08   5.73991425e-03   9.93922412e-01   3.37689999e-04]]
('unknown/out07262.png', array([2]))
[[  5.12969928e-05   6.99175969e-02   9.18765187e-01   1.12660211e-02]]
('unknown/out06368.png', array([2]))
[[  6.61348281e-07   1.11138411e-02   9.87771630e-01   1.11389079e-03]]
('unknown/out03166.png', array([2]))
[[  7.28552004e-06   3.74823548e-02   9.58224177e-01   4.28614626e-03]]
('unknown/out03198.png', array([2]))
[[  1.18381234e-11 

[[  4.84977178e-08   5.29030990e-03   9.94387329e-01   3.22360633e-04]]
('unknown/out03930.png', array([2]))
[[  1.22572374e-06   1.41051309e-02   9.84329760e-01   1.56392436e-03]]
('unknown/out02390.png', array([2]))
[[  2.29045381e-05   9.80347812e-01   4.74845013e-03   1.48808490e-02]]
('unknown/out00168.png', array([1]))
[[  2.12249185e-12   1.21149635e-04   9.99876976e-01   1.84815531e-06]]
('unknown/out05038.png', array([2]))
[[  3.96355778e-12   1.34450718e-04   9.99863148e-01   2.39825135e-06]]
('unknown/out00342.png', array([2]))
[[  2.77104206e-08   2.27567786e-03   9.97515440e-01   2.08898535e-04]]
('unknown/out06732.png', array([2]))
[[  1.41013197e-06   9.73572209e-03   9.88652527e-01   1.61036081e-03]]
('unknown/out01288.png', array([2]))
[[  1.60714844e-12   1.21086479e-04   9.99877214e-01   1.65111942e-06]]
('unknown/out02030.png', array([2]))
[[  1.04940838e-08   1.98082323e-03   9.97894108e-01   1.25056322e-04]]
('unknown/out07160.png', array([2]))
[[  8.46809911e-08 

In [9]:
print('Classification Report')
print(classification_report(test_generator.classes, pred_class_arr))

Classification Report
              precision    recall  f1-score   support

           1       0.94      1.00      0.97       291
           2       1.00      0.94      0.97       281
           3       1.00      0.95      0.97        55

   micro avg       0.97      0.97      0.97       627
   macro avg       0.98      0.96      0.97       627
weighted avg       0.97      0.97      0.97       627

